In [49]:
import pandas as pd
import ray 
import dask.dataframe as dd
import yaml
import logging
import os
import subprocess
import pandas as pd
import datetime 
import gc
import re
import modin.pandas as md

In [3]:
%%time
df=pd.read_csv("final.csv")

Wall time: 22.6 s


In [4]:
%%time
dfd=dd.read_csv("final.csv")

Wall time: 15 ms


In [10]:
df

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.100000,3.500000,1.400000,0.200000,Setosa
1,4.900000,3.000000,1.400000,0.200000,Setosa
2,4.700000,3.200000,1.300000,0.200000,Setosa
3,4.600000,3.100000,1.500000,0.200000,Setosa
4,5.000000,3.600000,1.400000,0.200000,Setosa
...,...,...,...,...,...
37415395,-5787.053379,3083.771796,806.658662,-1989.067365,Virginica
37415396,-5787.146746,3083.821449,806.671600,-1989.099445,Virginica
37415397,-5787.240113,3083.871102,806.684539,-1989.131524,Virginica
37415398,-5787.333481,3083.920755,806.697477,-1989.163604,Virginica


In [7]:
def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

In [33]:
%%writefile file.yaml
file_type: csv
dataset_name: iris
file_name: final
table_name: iris_data
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - sepal_length
    - sepal_width
    - petal_length
    - petal_width
    - variety

Overwriting file.yaml


In [34]:
import testutility as util
config_data = util.read_config_file("file.yaml")

In [35]:
config_data['inbound_delimiter']

','

In [36]:
config_data['columns']

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'variety']

In [37]:
config_data

{'file_type': 'csv',
 'dataset_name': 'iris',
 'file_name': 'final',
 'table_name': 'iris_data',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['sepal_length',
  'sepal_width',
  'petal_length',
  'petal_width',
  'variety']}

In [39]:
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [40]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'variety'],
      dtype='object')
columns of YAML are: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'variety']


In [41]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed


In [25]:
df.shape

(37415400, 5)

In [44]:
testdata = {
    'sepal_length' : [6.8,6.3,6.5,6.4],
    'sepal_width' : [4,4.5,3.2,3.1],
    'petal_length' : [1.5,1.2,2,5],
    'petal_width' : [0.5,1,2,3],
    'variety':['Setos','Setosa','Versicolor','Virginica']
}
dftest = pd.DataFrame(testdata, columns=['sepal_length', 'sepal_width','petal_length','petal_width','variety'])
dftest.to_csv("test_data.csv",index=False)